In [ ]:
# ?
!pip install pillow


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, SIFT
#from skimage.feature import plot_matches as plot_matches
from skimage.measure import ransac
from skimage.transform import ProjectiveTransform, warp
import os
from skimage import feature
import easyocr
from PIL import Image, ImageDraw, ImageFont


def load_sift(path):
    img_vzor = cv2.imread(path)
    # Convert to grayscale
    vzor_gray = cv2.cvtColor(img_vzor, cv2.COLOR_BGR2GRAY)
    # Initialize SIFT detector and extract keypoints and descriptors
    descriptor_extractor = SIFT()
    descriptor_extractor.detect_and_extract(vzor_gray)
    keypoints = descriptor_extractor.keypoints
    descriptors = descriptor_extractor.descriptors
    return keypoints, descriptors, img_vzor


def find_and_filter_matches(descriptors1, descriptors2, ratio=0.75):
    """
    Lowe's ratio test: keep match if d1 / d2 < ratio.
    match_descriptors does this when max_ratio is set.
    """
    matches = match_descriptors(
        descriptors1, descriptors2,
        metric='euclidean',
        cross_check=False,   # rely on the ratio test
        max_ratio=ratio
    )
    return matches


from skimage.measure import ransac
from skimage.transform import ProjectiveTransform

def estimate_transform(keypoints1, keypoints2, matches):
    # keypoints from skimage SIFT are (row, col) -> swap to (x,y) = (col,row)
    src = keypoints1[matches[:, 0]][:, ::-1]  # template
    dst = keypoints2[matches[:, 1]][:, ::-1]  # test

    model, inliers = ransac(
        (src, dst),
        ProjectiveTransform,
        min_samples=4,
        residual_threshold=3,
        max_trials=1000
    )
    return model, inliers



def extract_id_card(img_test, model, img_vzor):
    """
    img_test  ... test image (BGR)
    model     ... ProjectiveTransform from template -> test
    img_vzor  ... template image (BGR)

    Returns:
        aligned_card (BGR) in template coordinates
        test_with_box (BGR) test image with red polygon around card
    """
    h, w = img_vzor.shape[:2]

    # corners of template in (x, y)
    corners = np.array([[0, 0],
                        [w, 0],
                        [w, h],
                        [0, h]], dtype=float)

    # project corners into test image (template -> test)
    warped_corners = model(corners)
    warped_corners_int = np.round(warped_corners).astype(int)

    # draw red polygon on test image
    test_with_box = img_test.copy()
    cv2.polylines(
        test_with_box,
        [warped_corners_int.reshape(-1, 1, 2)],
        isClosed=True,
        color=(0, 0, 255),
        thickness=3
    )

    # homography matrix template->test, invert to get test->template
    H = model.params
    H_inv = np.linalg.inv(H)

    # warp TEST image into TEMPLATE coordinate system -> aligned card
    aligned_card = cv2.warpPerspective(img_test, H_inv, (w, h))

    return aligned_card, test_with_box


In [ ]:
keypoints_vzor, descriptors_vzor, img_vzor = load_sift("pvi_cv09/obcansky_prukaz_cr_sablona_2012_2014.png")
directory = "pvi_cv09/test/"

test_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if
              filename.endswith(('.jpg', '.png'))]
reader = easyocr.Reader(['cs', 'en'])

for path in test_paths:
    keypoints_vzor, descriptors_vzor, img_vzor = load_sift(
        "pvi_cv09/obcansky_prukaz_cr_sablona_2012_2014.png"
    )
directory = "pvi_cv09/test/"
test_paths = [os.path.join(directory, f)
              for f in os.listdir(directory)
              if f.lower().endswith(('.jpg', '.png'))]

for path in test_paths:
    keypoints_test, descriptors_test, img_test = load_sift(path)
    matches = find_and_filter_matches(descriptors_vzor, descriptors_test)

    model, inliers = estimate_transform(keypoints_vzor, keypoints_test, matches)
    inlier_matches = matches[inliers]

    # --- 1) show matches (left, still grayscale because plot_matches wants 2D) ---
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    feature.plot_matches(
        ax[0],
        rgb2gray(img_vzor),
        rgb2gray(img_test),
        keypoints_vzor,
        keypoints_test,
        inlier_matches,
        only_matches=True
    )
    ax[0].set_title("Filtered Matches Vzor–Test")
    ax[0].axis("off")

    # --- 2) extract & align ID card (color) ---
    aligned_bgr, test_with_box = extract_id_card(img_test, model, img_vzor)

    # show aligned card in COLOR (not gray)
    aligned_rgb = cv2.cvtColor(aligned_bgr, cv2.COLOR_BGR2RGB)
    ax[1].imshow(aligned_rgb)
    ax[1].set_title("Aligned ID Card")
    ax[1].axis("off")

    # show test image with red polygon
    ax[2].imshow(cv2.cvtColor(test_with_box, cv2.COLOR_BGR2RGB))
    ax[2].set_title("Test Image with Detected Card")
    ax[2].axis("off")

    plt.tight_layout()
    plt.show()


    # --- 3) OCR – read two lines from aligned card ---
    h, w = aligned_bgr.shape[:2]

    # PHOTO
    photo_y1 = int(0.25 * h)
    photo_y2 = int(0.90 * h)
    photo_x1 = int(0.05 * w)
    photo_x2 = int(0.35 * w)
    photo_roi = aligned_bgr[photo_y1:photo_y2, photo_x1:photo_x2]

    # ORIGINAL rectangle (unchanged)
    name_y1 = int(0.16 * h)
    name_y2 = int(0.35 * h)
    name_x1 = int(0.20 * w)
    name_x2 = int(0.50 * w)

    # --- SPLIT rectangle into two EQUAL horizontal halves ---
    mid_y = (name_y1 + name_y2) // 2

    upper_roi = aligned_bgr[name_y1:(int(mid_y*1.10)), name_x1:name_x2]
    lower_roi = aligned_bgr[int(mid_y*0.9):name_y2, name_x1:name_x2]

    # safety check
    if upper_roi.size == 0 or lower_roi.size == 0:
        print("ROI empty → adjust coordinates!")
        continue

    # Convert to RGB
    upper_rgb = cv2.cvtColor(upper_roi, cv2.COLOR_BGR2RGB)
    lower_rgb = cv2.cvtColor(lower_roi, cv2.COLOR_BGR2RGB)

    scale = 2.0  # nebo 3.0
    upper_big = cv2.resize(upper_roi, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    lower_big = cv2.resize(lower_roi, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)

    # hist equalization (optional)
    #gray = cv2.cvtColor(upper_roi, cv2.COLOR_BGR2GRAY)
    #gray2 = cv2.cvtColor(lower_roi, cv2.COLOR_BGR2GRAY)
    #upper_rgb = cv2.equalizeHist(gray)
    #lower_rgb = cv2.equalizeHist(gray2)

    # EasyOCR chce RGB
    upper_rgb = cv2.cvtColor(upper_big, cv2.COLOR_BGR2RGB)
    lower_rgb = cv2.cvtColor(lower_big, cv2.COLOR_BGR2RGB)

    upper_res = reader.readtext(upper_rgb, detail=0, allowlist='SPECIMEN')
    lower_res = reader.readtext(lower_rgb, detail=0, allowlist='VZOR')
    upper_text = " ".join(upper_res).strip()
    lower_text = " ".join(lower_res).strip()


    # OCR each half separately
    upper_text = " ".join(reader.readtext(upper_rgb, detail=0)).strip()
    lower_text = " ".join(reader.readtext(lower_rgb, detail=0)).strip()

    # --- 4) draw both lines onto the photo ---
    img_rgb = cv2.cvtColor(photo_roi, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img_rgb)
    draw = ImageDraw.Draw(pil_img)

    font = ImageFont.load_default()

    draw.text((10, 20), upper_text, font=font, fill=(0, 255, 0))
    draw.text((10, 40), lower_text, font=font, fill=(255, 255, 0))  # second line lower

    # Convert back
    photo_rgb = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

    # Debug rectangle
    debug = aligned_bgr.copy()
    cv2.rectangle(debug, (name_x1, name_y1), (name_x2, name_y2), (0, 0, 255), 2)

    plt.imshow(cv2.cvtColor(debug, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

    plt.figure(figsize=(4, 4))
    plt.title("ID Owner (photo + 2-line OCR)")
    plt.imshow(photo_rgb)
    plt.axis("off")
    plt.show()

    # --- DEBUG: visualize both split rectangles ---
    debug2 = aligned_bgr.copy()

    # full rectangle (optional)
    cv2.rectangle(debug2, (name_x1, name_y1), (name_x2, name_y2), (255, 0, 0), 2)

    # upper half – green
    cv2.rectangle(debug2,
                  (name_x1, name_y1),
                  (name_x2, int(mid_y*1.10)),
                  (0, 255, 0), 2)

    # lower half – yellow
    cv2.rectangle(debug2,
                  (name_x1, int(mid_y*0.9)),
                  (name_x2, name_y2),
                  (255, 255, 0), 2)

    plt.figure(figsize=(6, 6))
    plt.title("DEBUG: Split Name ROI")
    plt.imshow(cv2.cvtColor(debug2, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()
